### Importar libs e criar dic de amenities

In [2]:
import requests
import pandas as pd
from time import sleep
from tqdm import tqdm
from datetime import datetime
from utils import config
import ast

In [3]:
_AMENITIES = {
        "Kitchen"                       : [2, 8],
        "Wifi"                          : 4,
        "Air conditioning"              : 5,
        "Pool"                          : 7,
        "Free parking on premises"      : 9,
        "Smoking allowed"               : 11,
        "Pets allowed"                  : 12,
        "Gym"                           : 15,
        "Breakfast"                     : 16,
        "Elevator"                      : 21,
        "Hot tub"                       : 25,
        "Indoor fireplace"              : 27,
        "Heating"                       : 30,
        "Washer"                        : 33,
        "Dryer"                         : 34,
        "Smoke alarm"                   : 35,
        "Carbon monoxide alarm"         : 36,
        "Shampoo"                       : 41,
        "Hangers"                       : 44,
        "Hair dryer"                    : 45,
        "Iron"                          : 46,
        "Laptop-friendly workspace"     : 47,
        "Self check-in"                 : 51,
        "TV"                            : 58,
        "High chair"                    : 64,
        "Private bathroom"              : 78,
        "Wide hallways"                 : 109,
        "No stairs or steps to enter"   : [110, 115, 120, 127],
        "Wide entrance for guests"      : 111,
        "Step-free path to entrance"    : 112,
        "Well-lit path to entrance"     : 113,
        "Disabled parking spot"         : 114,
        "Wide entrance"                 : 116,
        "Extra space around bed"        : 117,
        "Accessible-height bed"         : 118,
        "Wide doorway to guest bathroom": 121,
        "Bathtub with bath chair"       : 123,
        "Accessible-height toilet"      : 125,
        "Wide entryway"                 : 128,
        "Handheld shower head"          : 136,
        "Crib"                          : 286,
        "Electric profiling bed"        : 288,
        "Mobile hoist"                  : 289,
        "Pool with pool hoist"          : 290,
        "Ceiling hoist"                 : 291,
        "Fixed grab bars for shower"    : 294,
        "Fixed grab bars for toilet"    : 295,
        "Step-free shower"              : 296,
        "Shower chair"                  : 297,
        "Piano"                         : 347,
        "Extra space around toilet"     : 608,
        "Extra space around shower"     : 609
    }

_AMENITIES = {k.lower() : v for k, v in _AMENITIES.items()}

### Função de Busca com Parâmetros inseridos pelo usuário

In [4]:
def invert_date(date):
    '''Convert date format from YYYY-MM-DD to DD-MM-YY'''
    
    date = date.split('-')[::-1]
    y = date.pop(-1)[2:]
    date.append(y)
    return '-'.join(date)

def search(city, checkin, checkout, adults=1, childrens=0, infants=0): # 3 parâmentros obrigatórios e 3 opcionais
    '''Use RapidAPI's Airbnb API to search for accommodations based on given parameters, print the average total price and returns a DataFrame with all accommodations found'''

    headers = {
    "X-RapidAPI-Key" : config.API_KEY, # token e host fornecidos pelo API na hora do cadastro
    "X-RapidAPI-Host": config.API_HOST
    }

    url = "https://airbnb13.p.rapidapi.com/search-location" # url base da API

    # lista de parâmentros para a pesquisa.    
    querystring = {"location" : city,
                   "checkin" : checkin,
                   "checkout": checkout,
                   "adults"  : adults,
                   "children": childrens,
                   "infants" : infants,
                   "page"    : "1",
                   "currency": "BRL"}

    # Laço para aquisição dos resultados (maxímo de 8 paginas, com 40 resultados por página. limite da API)        
    results = []
    for page in tqdm(range(1, 9)):
        querystring["page"] = str(page)
        response = requests.get(url, headers=headers, params=querystring).json()
        if response.get("error") == True or response.get("message") != None:
            print(response.get("message"))
            break
        else:
            results.extend(response["results"])
        sleep(12.5) # A API só deixa fazer 5 requests por minutos.

    # Criação de um DF com o resultado total encontrado
    df = pd.DataFrame(data=results)

    # Criar arquivo csv
    main_city = df.city.value_counts().index[0]
    df.to_csv(f'../data/raw/{main_city}_{checkin}_{checkout}_raw.csv', sep=';', index=False)

    # Imprimir quantidade de acomodações encontradas e média de preço do período escolhido
    print(f"{len(results)} results were found.")    
    datein = invert_date(checkin)
    dateout = invert_date(checkout)    
    sum = 0
    for ids in results:
        sum += ids["price"]["total"]
    print(f"Avarege price for {datein} to {dateout}: {sum/(int(len(results))) :.2f} {querystring['currency']}")

    return df

### Inserção das variáveis para a pesquisa no site

#### Testa data CHECKIN

In [5]:
def checkin():
    '''Verify if check-in date is today or after and return it formatted for API's use'''
    
    atual_date = datetime.today()
    var = input("Checkin (Format: dd/mm/aaaa) *Required: ")
    date = datetime.strptime(var,"%d/%m/%Y")
    checkin_date = datetime.strftime(date, "%Y-%m-%d")
    if date < atual_date:
        print("Date can not be in the past!")
        checkin_date = checkin()
    return checkin_date

#### Testa data CHECKOUT

In [6]:
def checkout(checkin_date):
    '''Verify if check-out date is after check-in date and return it formatted for API's use'''
    
    var = input("Checkout (Format: dd/mm/aaaa) *Required: ")
    date = datetime.strptime(var,"%d/%m/%Y")
    checkin_date = datetime.strptime(checkin_date,"%Y-%m-%d")
    checkout_date = datetime.strftime(date,"%Y-%m-%d")
    if date <= checkin_date:
        print("Checkout must be after checkin")
        checkout_date = checkout()
    return checkout_date

### Entrada dos parâmetros de pesquisa

In [6]:
city = input("Enter the City *Required: ")
checkin = checkin()
checkout = checkout(checkin)
adults = input("Adults (+13y) *Default = 1): ")
childrens = input("Childerns (2-12y) *Default = 0): ")
infants = input("Infants (U-2y) *Default = 0): ")

df = search(city, checkin, checkout, adults, childrens, infants) # Chamada da função com os parâmentros

100%|██████████| 8/8 [02:10<00:00, 16.34s/it]

300 results were found.
Avarege price for 12-05-23 to 01-06-23: 13305.06 BRL


### Função Filtro

In [7]:
def filter(csv_path, min_bathrooms : float = None, min_bedrooms : int = None, min_beds : int = None, city_only : bool = False, 
           is_superhost : bool = False, rarefind : bool = False, min_person : int = None, min_rvw : int = None, 
           types : str | list = None, max_night_rate : int = None, max_price : int = None, min_rating : float = None, amenities : list = None) -> pd.DataFrame:
    '''Filter API response with based given parameters, print min/max/avg night's rate and min/max/avg total price for the stay and return filtered Dataframe'''

    # Criar DF com csv_path
    df = pd.read_csv(csv_path, sep=';')

    # Arrumar tipos das colunas
    df.price = df.price.apply(lambda price: ast.literal_eval(price))
    df.amenityIds = df.amenityIds.apply(lambda x: ast.literal_eval(x))

    # Copiar df para preservar o original
    df_filter = df.copy()

    # Definir currency
    currency = df_filter.price[0]['currency']

    # Filtrar parâmetros opcionais
    if min_bathrooms:
        df_filter = df_filter.loc[df_filter.bathrooms >= min_bathrooms].copy()
    if min_bedrooms:
        df_filter = df_filter.loc[df_filter.bedrooms >= min_bedrooms].copy()
    if min_beds:
        df_filter = df_filter.loc[df_filter.beds >= min_beds].copy()
    if city_only:
        df_filter = df_filter.loc[df_filter.city == df_filter.city.value_counts().index[0]].copy()
    if is_superhost:
        df_filter = df_filter.loc[df_filter.isSuperhost == True].copy()
    if rarefind:
        df_filter = df_filter.loc[df_filter.rareFind == True].copy()
    if min_person:
        df_filter = df_filter.loc[df_filter.persons >= min_person].copy()
    if min_rvw:
        df_filter = df_filter.loc[df_filter.reviewsCount >= min_rvw].copy()

    # Filtrar types
    if types:
        if type(types) is str:
            if types.lower() in [t.lower() for t in df_filter.type.unique()]:
                df_filter = df_filter.loc[df_filter.type.apply(str.lower) == types.lower()].copy()
            else:
                return print(f'Type not available, please enter a valid type among the following:\n{df.type.unique()}')
        if type(types) is list:
            types = [i.lower() for i in types]
            check_types = [value in df_filter.type.apply(str.lower).unique() for value in types]
            if any(check_types):
                df_filter = df_filter.loc[df_filter.type.map(lambda value: value.lower() in types)].copy()
            else:
                return print(f'Types not available, please enter a valid type among the following:\n{df.type.unique()}')
            if not all(check_types):
                for value in types:
                    if value not in df_filter.type.apply(str.lower).unique():
                        print(f'{value} is not a valid type and was not used in this filter.')
                print('Please use the following list of types:', df.type.unique(), sep='\n')
    if max_night_rate:
        df_filter = df_filter.loc[df_filter.price.map(lambda price: price['rate'] <= max_night_rate)].copy()
    if max_price:
        df_filter = df_filter.loc[df_filter.price.map(lambda price: price['total'] <= max_price)].copy()
    if min_rating:
        df_filter = df_filter.loc[df_filter.rating >= min_rating].copy()
    
    # Filtrar amenities
    if amenities:
        wrong_amenity = []
        if type(amenities) is str:
            amenities = [amenities]
        if type(amenities) is list:
            amenities = {a : _AMENITIES.get(a.lower(), False) for a in amenities}
        for amenity, code in amenities.items():
            if type(code) is int:
                df_filter = df_filter.loc[df_filter.amenityIds.map(lambda x: code in x)].copy()
            elif type(code) is list:
                df_filter = df_filter.loc[df_filter.amenityIds.map(lambda x: any([c in x for c in code]))].copy()
            else:
                wrong_amenity.append(amenity)
        if len(wrong_amenity) > 0:
            print(f'The following amenities are not available:\n{wrong_amenity}\nPlease enter a valid amenity among the following:\n{list(_AMENITIES.keys())}')

    # Printar resultados
    total = len(df_filter)
    print(f'\nFound {total} accommodations:')
    if total > 0:
        print('-' * 30)
        print('Minimum night rate:', df_filter.price.map(lambda value: value['rate']).min(), currency)
        print('Maximum night rate:', df_filter.price.map(lambda value: value['rate']).max(), currency)
        l = print('Average night rate:', round(df_filter.price.map(lambda value: value['rate']).mean(), 2), currency)
        print('-' * 30)
        print('Minimum total price:', df_filter.price.map(lambda value: value['total']).min(), currency)
        print('Maximum total price:', df_filter.price.map(lambda value: value['total']).max(), currency)
        print('Average total price:', round(df_filter.price.map(lambda value: value['total']).mean(), 2), currency)
        print('-' * 30)
        
    return df_filter

def show_filter_links(df_filter):
    '''Print all deeplinks in filtered DataFrame, for quick access'''

    for link in df_filter.deeplink:
        print(link)

def save_filtered(df_filter):
    '''Save filtered DataFrame as csv file in pre-defined directory and print confirmation message'''

    df_filter.to_csv(f'../data/filtered/{city}_{checkin}_{checkout}_filtered.csv', sep=';', index=False)
    print('CSV File Saved to directory: ../data/filtered/')
    

In [20]:
df_filter = filter('../data/raw/Toronto_2023-05-12_2023-06-01_raw.csv', min_bathrooms=None, 
                    min_bedrooms=None, min_beds=None, city_only=False, 
                    is_superhost=False, rarefind=False, min_person=None, min_rvw=None, 
                    types=None, max_night_rate=None, max_price=10000, min_rating=None, amenities='pets allowed')

show_filter_links(df_filter)


Found 13 accommodations:
------------------------------
Minimum night rate: 202 BRL
Maximum night rate: 497 BRL
Average night rate: 373.23 BRL
------------------------------
Minimum total price: 4031 BRL
Maximum total price: 9935 BRL
Average total price: 7458.31 BRL
------------------------------
https://www.airbnb.com/rooms/42176804?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/44063885?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/712779081548499784?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/28226625?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/687873959079901011?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/44684660?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/729217536286267523?check_in=2023-05-12&check_out=2023-06-01&adults=2
https://www.airbnb.com/rooms/25937728?check_in=2023-05-

In [9]:
save_filtered(df_filter)

CSV File Saved to directory: ../data/filtered/


### Regressão

In [51]:
df = pd.read_csv('../data/raw/Toronto_2023-05-12_2023-06-01_raw.csv', sep=';')
df.amenityIds = df.amenityIds.apply(lambda x: ast.literal_eval(x))
df.head()

,id,url,deeplink,position,name,bathrooms,bedrooms,beds,city,images,...,persons,reviewsCount,rating,type,userId,address,amenityIds,previewAmenities,cancelPolicy,price
0,50035018,https://www.airbnb.com/rooms/50035018,https://www.airbnb.com/rooms/50035018?check_in...,1,FREE parking-QUEEN bed-GROUND level! Cozy room,1.0,1,1.0,Toronto,['https://a0.muscache.com/im/pictures/666f66dc...,...,2,176,4.82,Private room in home,211220735,"Toronto, ON, Canada","[4, 5, 8, 9, 73, 137, 77, 79, 16, 657, 146, 21...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_FLEXIBLE,"{'rate': 262, 'currency': 'BRL', 'total': 5228..."
1,623302088682592936,https://www.airbnb.com/rooms/623302088682592936,https://www.airbnb.com/rooms/62330208868259293...,2,Lovely One Bedroom Apartment in Roncesvalles,1.0,1,1.0,Toronto,['https://a0.muscache.com/im/pictures/miso/Hos...,...,3,24,4.96,Entire condo,34610851,"Toronto, ON, Canada","[1, 4, 5, 8, 137, 10, 522, 139, 657, 280, 665,...","['Wifi', 'Kitchen', 'Self check-in']",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 1043, 'currency': 'BRL', 'total': 208..."
2,855851545248241829,https://www.airbnb.com/rooms/855851545248241829,https://www.airbnb.com/rooms/85585154524824182...,3,Cozy bedroom in a peaceful neighbourhood.,1.5,1,1.0,Mississauga,['https://a0.muscache.com/im/pictures/miso/Hos...,...,2,0,NaN,Private room in home,455017314,"Mississauga, ON, Canada","[1, 4, 5, 8, 9, 137, 522, 657, 663, 665, 30, 4...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_FLEXIBLE,"{'rate': 258, 'currency': 'BRL', 'total': 5143..."
3,647763850981043911,https://www.airbnb.com/rooms/647763850981043911,https://www.airbnb.com/rooms/64776385098104391...,4,"Lovely, clean 2-bedroom condo in Oakville",1.5,2,2.0,Oakville,['https://a0.muscache.com/im/pictures/bef68507...,...,4,8,5.00,Entire condo,463883978,"Oakville, ON, Canada","[1, 4, 5, 8, 9, 137, 12, 657, 23, 280, 665, 30...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 594, 'currency': 'BRL', 'total': 1186..."
4,40395497,https://www.airbnb.com/rooms/40395497,https://www.airbnb.com/rooms/40395497?check_in...,5,✨Toronto Cozy Guest Suite✨💫🏠🧿,1.0,1,2.0,Toronto,['https://a0.muscache.com/im/pictures/387e4cef...,...,2,53,4.89,Entire guest suite,312250265,"Toronto, ON, Canada","[1, 4, 8, 10, 77, 79, 85, 86, 280, 89, 90, 91,...","['Wifi', 'Kitchen', 'Self check-in']",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 625, 'currency': 'BRL', 'total': 1249..."


In [52]:
for column, value in list(_AMENITIES.items())[::-1]:
    if type(value) == int:
        value = [value]
    values = [any(item in value for item in row['amenityIds']) for _, row in df.iterrows()]
    df.insert(17, column, values)

In [54]:
df

,id,url,deeplink,position,name,bathrooms,bedrooms,beds,city,images,...,extra space around toilet,extra space around shower,rating,type,userId,address,amenityIds,previewAmenities,cancelPolicy,price
0,50035018,https://www.airbnb.com/rooms/50035018,https://www.airbnb.com/rooms/50035018?check_in...,1,FREE parking-QUEEN bed-GROUND level! Cozy room,1.0,1,1.0,Toronto,['https://a0.muscache.com/im/pictures/666f66dc...,...,False,False,4.82,Private room in home,211220735,"Toronto, ON, Canada","[4, 5, 8, 9, 73, 137, 77, 79, 16, 657, 146, 21...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_FLEXIBLE,"{'rate': 262, 'currency': 'BRL', 'total': 5228..."
1,623302088682592936,https://www.airbnb.com/rooms/623302088682592936,https://www.airbnb.com/rooms/62330208868259293...,2,Lovely One Bedroom Apartment in Roncesvalles,1.0,1,1.0,Toronto,['https://a0.muscache.com/im/pictures/miso/Hos...,...,False,False,4.96,Entire condo,34610851,"Toronto, ON, Canada","[1, 4, 5, 8, 137, 10, 522, 139, 657, 280, 665,...","['Wifi', 'Kitchen', 'Self check-in']",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 1043, 'currency': 'BRL', 'total': 208..."
2,855851545248241829,https://www.airbnb.com/rooms/855851545248241829,https://www.airbnb.com/rooms/85585154524824182...,3,Cozy bedroom in a peaceful neighbourhood.,1.5,1,1.0,Mississauga,['https://a0.muscache.com/im/pictures/miso/Hos...,...,False,False,NaN,Private room in home,455017314,"Mississauga, ON, Canada","[1, 4, 5, 8, 9, 137, 522, 657, 663, 665, 30, 4...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_FLEXIBLE,"{'rate': 258, 'currency': 'BRL', 'total': 5143..."
3,647763850981043911,https://www.airbnb.com/rooms/647763850981043911,https://www.airbnb.com/rooms/64776385098104391...,4,"Lovely, clean 2-bedroom condo in Oakville",1.5,2,2.0,Oakville,['https://a0.muscache.com/im/pictures/bef68507...,...,False,False,5.00,Entire condo,463883978,"Oakville, ON, Canada","[1, 4, 5, 8, 9, 137, 12, 657, 23, 280, 665, 30...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_BETTER_STRICT_WITH_GRACE_PERIOD,"{'rate': 594, 'currency': 'BRL', 'total': 1186..."
4,40395497,https://www.airbnb.com/rooms/40395497,https://www.airbnb.com/rooms/40395497?check_in...,5,✨Toronto Cozy Guest Suite✨💫🏠🧿,1.0,1,2.0,Toronto,['https://a0.muscache.com/im/pictures/387e4cef...,...,False,False,4.89,Entire guest suite,312250265,"Toronto, ON, Canada","[1, 4, 8, 10, 77, 79, 85, 86, 280, 89, 90, 91,...","['Wifi', 'Kitchen', 'Self check-in']",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 625, 'currency': 'BRL', 'total': 1249..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,647097741526033030,https://www.airbnb.com/rooms/647097741526033030,https://www.airbnb.com/rooms/64709774152603303...,296,"Centrally Located Unit - WIFI, Workspace & Par...",1.0,2,2.0,Toronto,['https://a0.muscache.com/im/pictures/miso/Hos...,...,False,False,4.78,Entire guest suite,463110139,"Toronto, ON, Canada","[1, 4, 5, 8, 9, 137, 394, 522, 12, 657, 23, 66...","['Free parking', 'Wifi', 'Kitchen', 'Self chec...",CANCEL_STRICT_14_WITH_GRACE_PERIOD,"{'rate': 1202, 'currency': 'BRL', 'total': 240..."
296,21816249,https://www.airbnb.com/rooms/21816249,https://www.airbnb.com/rooms/21816249?check_in...,297,Get Comfortable in Our Stylish Place,1.0,1,1.0,Toronto,['https://a0.muscache.com/im/pictures/monet/Se...,...,False,False,4.94,Private room in bungalow,158776724,"Toronto, ON, Canada","[1, 2, 4, 5, 9, 137, 265, 140, 77, 79, 81, 657...","['Free parking', 'Wifi', 'Self check-in']",CANCEL_MODERATE,"{'rate': 488, 'currency': 'BRL', 'total': 9760..."
297,754536492407971375,https://www.airbnb.com/rooms/754536492407971375,https://www.airbnb.com/rooms/75453649240797137...,298,Boho in the burbs- Secluded Forest Backyard,1.5,2,2.0,Aurora,['https://a0.muscache.com/im/pictures/miso/Hos...,...,False,False,5.00,Entire home,49877896,"Aurora, ON, Canada","[1, 4, 5, 8, 9, 137, 12, 145, 657, 280, 665, 3...","['Free parking', 'Wifi', 'Kitch